In [1]:
import pandas as pd
import keras
import numpy as np

# Import CIFAR 10 dataset
from keras.datasets import cifar10

from keras.preprocessing.image import ImageDataGenerator

# Import Necessary CNN Building Blocks
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Activation, Dropout, MaxPool2D, BatchNormalization, GlobalAveragePooling2D, UpSampling2D
from keras.layers.advanced_activations import LeakyReLU

from keras import backend as K
from keras import applications
from keras import callbacks
from keras import optimizers
from keras.utils import np_utils

import tensorflow as tf
import tensorflow_addons as tfa

from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
# Requirements:
# !pip install keras
# !pip install tensorflow
# !pip install keras
# !pip install tensorflow_addons

# TODO: data augmentation validation

## Data

In [25]:
# Load Dataset
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

BATCH_SIZE = 32
INPUT_SHAPE = X_train.shape[1:]

In [ ]:
# TODO: Configure data augmentation
# train_datagen = ImageDataGenerator(
#     rescale=1./255,
#     shear_range=0.2,
#     zoom_range=0.2,
#     horizontal_flip=True)

# train_datagen.fit(X_train)
# train_generator = train_datagen.flow(X_train, y_train, batch_size=BATCH_SIZE)

# test_datagen = ImageDataGenerator(rescale=1./255)
# validation_generator = test_datagen.flow(X_test, y_test, batch_size=BATCH_SIZE)

## Fitting the Deep Learning Models

### Custom Models

In [26]:
# Normalize input data
X_train_norm = X_train/255
X_test_norm = X_test/255

# Convert class labels to one-hot encoded
y_train_ohe = keras.utils.np_utils.to_categorical(y_train)
y_test_ohe = keras.utils.np_utils.to_categorical(y_test)

tqdm_callback = tfa.callbacks.TQDMProgressBar()

LR=3e-3
EPOCHS=30  # 200

In [5]:
# Scheduler of learning rate (decay with epochs)
def lrScheduler(epoch):
    return LR * 0.9 ** epoch

#### Model concept no. 1

In [6]:
model1 = Sequential()

model1.add(Conv2D(filters = 16, kernel_size = (3, 3), padding='same', input_shape=INPUT_SHAPE))
model1.add(LeakyReLU(0.1))
model1.add(Conv2D(filters = 32, kernel_size = (3, 3), padding='same'))
model1.add(LeakyReLU(0.1))
model1.add(MaxPooling2D((2, 2)))
model1.add(Dropout(0.25))
model1.add(Conv2D(filters = 32, kernel_size = (3, 3), padding='same'))
model1.add(LeakyReLU(0.1))
model1.add(Conv2D(filters = 64, kernel_size = (3, 3), padding='same'))
model1.add(LeakyReLU(0.1))
model1.add(MaxPooling2D((2, 2)))
model1.add(Dropout(0.25))
model1.add(Flatten())
model1.add(Dense(256))
model1.add(LeakyReLU(0.1))
model1.add(Dropout(0.5))
model1.add(Dense(10))
model1.add(Activation("softmax"))

model1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 32, 32, 16)        448       
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 32, 32, 16)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 32, 32, 32)        4640      
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 32, 32, 32)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 16, 16, 32)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 16, 16, 32)        0         
                                                        

In [7]:
model1.compile(
    loss='categorical_crossentropy',  
    optimizer=tf.keras.optimizers.Adam(learning_rate=LR), 
    metrics=['accuracy']
)

In [9]:
model1.fit(
    X_train_norm,
    y_train_ohe,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    callbacks=[keras.callbacks.LearningRateScheduler(lrScheduler), 
               tqdm_callback,],
    shuffle=True,
    verbose=1,
    validation_data=(X_test_norm, y_test_ohe)
)

Training:   0%|                                                                                0/30 ETA: ?s,  …

Epoch 1/30


0/1563                                                                                                       E…

Epoch 1/30
1563/1563 [==============================] - 102s 65ms/step - loss: 1.5564 - accuracy: 0.4372 - val_loss: 1.1555 - val_accuracy: 0.5859 - lr: 0.0030
Epoch 2/30


0/1563                                                                                                       E…

...

Epoch 28/30
1563/1563 [==============================] - 107s 69ms/step - loss: 0.3252 - accuracy: 0.8834 - val_loss: 0.6294 - val_accuracy: 0.8072 - lr: 1.7445e-04
Epoch 29/30


0/1563                                                                                                       E…

Epoch 29/30
1563/1563 [==============================] - 103s 66ms/step - loss: 0.3172 - accuracy: 0.8868 - val_loss: 0.6146 - val_accuracy: 0.8098 - lr: 1.5700e-04
Epoch 30/30


0/1563                                                                                                       E…

Epoch 30/30
1563/1563 [==============================] - 107s 68ms/step - loss: 0.3128 - accuracy: 0.8859 - val_loss: 0.6153 - val_accuracy: 0.8120 - lr: 1.4130e-04


#### Model concept no. 2

In [15]:
EPOCHS=200
PATIENCE=20

early_stop = EarlyStopping(monitor='loss', patience=PATIENCE)

In [16]:
model2 = Sequential()

model2.add(Conv2D(filters=32, kernel_size=(3,3), input_shape=INPUT_SHAPE, activation='relu', padding = 'same', kernel_initializer='he_uniform'))
model2.add(Conv2D(filters=32, kernel_size=(3,3), input_shape=INPUT_SHAPE, activation='relu', padding = 'same', kernel_initializer='he_uniform'))
model2.add(MaxPooling2D(pool_size=(2, 2)))
model2.add(Dropout(0.3))
model2.add(Conv2D(filters=64, kernel_size=(3,3), input_shape=INPUT_SHAPE, activation='relu', padding = 'same', kernel_initializer='he_uniform'))
model2.add(Conv2D(filters=64, kernel_size=(3,3), input_shape=INPUT_SHAPE, activation='relu', padding = 'same', kernel_initializer='he_uniform'))
model2.add(MaxPooling2D(pool_size=(2, 2)))
model2.add(Dropout(0.3))
model2.add(Conv2D(filters=128, kernel_size=(3,3), input_shape=INPUT_SHAPE, activation='relu', padding = 'same', kernel_initializer='he_uniform'))
model2.add(Conv2D(filters=128, kernel_size=(3,3), input_shape=INPUT_SHAPE, activation='relu', padding = 'same', kernel_initializer='he_uniform'))
model2.add(MaxPooling2D(pool_size=(2, 2)))
model2.add(Dropout(0.3))
model2.add(BatchNormalization())
model2.add(Flatten())
model2.add(Dense(256, activation='relu', kernel_initializer='he_uniform'))
model2.add(Dropout(0.4))
model2.add(Dense(10, activation='softmax'))

In [17]:
model2.compile(loss='categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(learning_rate=LR), 
              metrics=['accuracy'])

In [18]:
model2.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_10 (Conv2D)          (None, 32, 32, 32)        896       
                                                                 
 conv2d_11 (Conv2D)          (None, 32, 32, 32)        9248      
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 16, 16, 32)       0         
 2D)                                                             
                                                                 
 dropout_7 (Dropout)         (None, 16, 16, 32)        0         
                                                                 
 conv2d_12 (Conv2D)          (None, 16, 16, 64)        18496     
                                                                 
 conv2d_13 (Conv2D)          (None, 16, 16, 64)        36928     
                                                      

In [19]:
model2.fit(
    X_train_norm,
    y_train_ohe,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    callbacks=[
       keras.callbacks.LearningRateScheduler(lrScheduler), 
       tqdm_callback,
       early_stop],
    shuffle=True,
    verbose=1,
    validation_data=(X_test_norm, y_test_ohe)
    )

Training:   0%|                                                                               0/200 ETA: ?s,  …

Epoch 1/200


0/1563                                                                                                       E…

Epoch 1/200
1563/1563 [==============================] - 196s 125ms/step - loss: 1.9610 - accuracy: 0.2554 - val_loss: 1.7258 - val_accuracy: 0.3252 - lr: 0.0030
Epoch 2/200


0/1563                                                                                                       E…

...

Epoch 85/200
1563/1563 [==============================] - 166s 106ms/step - loss: 0.3610 - accuracy: 0.8720 - val_loss: 0.5248 - val_accuracy: 0.8297 - lr: 4.3002e-07
Epoch 86/200


0/1563                                                                                                       E…

Epoch 86/200
1563/1563 [==============================] - 169s 108ms/step - loss: 0.3655 - accuracy: 0.8689 - val_loss: 0.5261 - val_accuracy: 0.8298 - lr: 3.8702e-07
Epoch 87/200


0/1563                                                                                                       E…

Epoch 87/200
1563/1563 [==============================] - 168s 108ms/step - loss: 0.3626 - accuracy: 0.8702 - val_loss: 0.5253 - val_accuracy: 0.8297 - lr: 3.4832e-07


#### Model concept no. 3

In [20]:
BATCH_SIZE=128
EPOCHS=20

In [21]:
model3 = Sequential()

model3.add(Conv2D(filters=32, kernel_size=(4,4), input_shape=INPUT_SHAPE, activation='relu',))
model3.add(MaxPool2D(pool_size=(2, 2)))
model3.add(Conv2D(filters=32, kernel_size=(4,4), activation='relu',))
model3.add(MaxPool2D(pool_size=(2, 2)))
model3.add(Flatten())
model3.add(Dense(128, activation='relu'))
model3.add(Dropout(0.2))
model3.add(Dense(64, activation='relu'))
model3.add(Dropout(0.2))
model3.add(Dense(32, activation='relu'))
model3.add(Dropout(0.2))
model3.add(Dense(10, activation='softmax'))

In [22]:
model3.compile(loss='categorical_crossentropy',
               optimizer=tf.keras.optimizers.Adam(learning_rate=LR), 
               metrics=['accuracy'])

In [23]:
model3.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_16 (Conv2D)          (None, 29, 29, 32)        1568      
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 14, 14, 32)       0         
 2D)                                                             
                                                                 
 conv2d_17 (Conv2D)          (None, 11, 11, 32)        16416     
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 5, 5, 32)         0         
 2D)                                                             
                                                                 
 flatten_3 (Flatten)         (None, 800)               0         
                                                                 
 dense_6 (Dense)             (None, 128)              

In [24]:
model3.fit(
    X_train_norm,
    y_train_ohe,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    callbacks=[
      keras.callbacks.LearningRateScheduler(lrScheduler), 
       tqdm_callback,],
    shuffle=True,
    verbose=1,
    validation_data=(X_test_norm, y_test_ohe)
    )

Training:   0%|                                                                                0/20 ETA: ?s,  …

Epoch 1/20


0/391                                                                                                        E…

Epoch 1/20
391/391 [==============================] - 26s 65ms/step - loss: 1.8884 - accuracy: 0.2902 - val_loss: 1.5805 - val_accuracy: 0.4187 - lr: 0.0030
Epoch 2/20


0/391                                                                                                        E…

...

Epoch 18/20
391/391 [==============================] - 24s 63ms/step - loss: 1.0037 - accuracy: 0.6520 - val_loss: 1.1258 - val_accuracy: 0.6097 - lr: 5.0032e-04
Epoch 19/20


0/391                                                                                                        E…

Epoch 19/20
391/391 [==============================] - 24s 63ms/step - loss: 0.9940 - accuracy: 0.6535 - val_loss: 1.1272 - val_accuracy: 0.6099 - lr: 4.5028e-04
Epoch 20/20


0/391                                                                                                        E…

Epoch 20/20
391/391 [==============================] - 24s 62ms/step - loss: 0.9833 - accuracy: 0.6578 - val_loss: 1.1231 - val_accuracy: 0.6125 - lr: 4.0526e-04


### Pretrained Models

#### Residual Network (ResNet)

https://medium.com/@kenneth.ca95/a-guide-to-transfer-learning-with-keras-using-resnet50-a81a4a28084b

In [26]:
LR = 2e-5
EPOCHS=10

In [27]:
def preprocess_data(X, y):
    """
    a function that trains a convolutional neural network to classify the CIFAR 10 dataset
    
    :param X: X is a numpy.ndarray of shape(m, 32, 32, 3) containing the CIFAR 10 data,
    where m is the number of data points
    :param y: y is a numpy.ndarray of shape(m,) containing the CIFAR 10 data,
    labels for X
    :return: X_p, y_p
        X_p is a numpy.ndarray containing the preprocessed X
        y_p is a numpy.ndarray containing the preprocessed y
    """
    X_p = tf.keras.applications.resnet50.preprocess_input(X)
    y_p = tf.keras.utils.to_categorical(y, 10)
    return X_p, y_p

In [28]:
X_train_resnet50, y_train_resnet50 = preprocess_data(X_train, y_train)
X_test_resnet50, y_test_resnet50 = preprocess_data(X_test, y_test)

input_tensor=tf.keras.Input(shape=X_train_resnet50.shape[1:])

In [29]:
res_model = tf.keras.applications.resnet50.ResNet50(weights='imagenet', include_top=False, input_tensor=input_tensor)

for layer in res_model.layers[:143]:
    layer.trainable=False
    
resNet = Sequential()
resNet.add(res_model)
resNet.add(tf.keras.layers.Flatten())
resNet.add(tf.keras.layers.Dense(10, activation='softmax'))

In [30]:
resNet.compile(loss='categorical_crossentropy',
               optimizer=tf.keras.optimizers.RMSprop(learning_rate=LR), 
               metrics=['accuracy'])

In [31]:
resNet.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 1, 1, 2048)        23587712  
                                                                 
 flatten_4 (Flatten)         (None, 2048)              0         
                                                                 
 dense_10 (Dense)            (None, 10)                20490     
                                                                 
Total params: 23,608,202
Trainable params: 14,996,490
Non-trainable params: 8,611,712
_________________________________________________________________


In [32]:
resNet.fit(X_train_resnet50,
           y_train_resnet50,
           batch_size=BATCH_SIZE,
           epochs=EPOCHS,
           shuffle=True,
           verbose=1,
           validation_data=(X_test_resnet50, y_test_resnet50)
          )

Epoch 1/10
391/391 [==============================] - 982s 3s/step - loss: 1.9412 - accuracy: 0.4174 - val_loss: 1.4056 - val_accuracy: 0.5417
Epoch 2/10
391/391 [==============================] - 988s 3s/step - loss: 1.0850 - accuracy: 0.6395 - val_loss: 1.2232 - val_accuracy: 0.6122
Epoch 3/10
391/391 [==============================] - 980s 3s/step - loss: 0.7825 - accuracy: 0.7352 - val_loss: 1.1423 - val_accuracy: 0.6361
Epoch 4/10
391/391 [==============================] - 982s 3s/step - loss: 0.5872 - accuracy: 0.8035 - val_loss: 1.1092 - val_accuracy: 0.6509
Epoch 5/10
391/391 [==============================] - 979s 3s/step - loss: 0.4388 - accuracy: 0.8618 - val_loss: 1.1027 - val_accuracy: 0.6570
Epoch 6/10
391/391 [==============================] - 996s 3s/step - loss: 0.3233 - accuracy: 0.9075 - val_loss: 1.0933 - val_accuracy: 0.6647
Epoch 7/10
391/391 [==============================] - 982s 3s/step - loss: 0.2350 - accuracy: 0.9405 - val_loss: 1.1156 - val_accuracy: 0.6650

#### VGG16

In [15]:
EPOCHS=10  # 50
BATCH_SIZE = 32
LR=1e-3
MOMENTUM=0.9

In [16]:
def preprocess_data(X, y):
    """
    a function that trains a convolutional neural network to classify the CIFAR 10 dataset
    
    :param X: X is a numpy.ndarray of shape(m, 32, 32, 3) containing the CIFAR 10 data,
    where m is the number of data points
    :param y: y is a numpy.ndarray of shape(m,) containing the CIFAR 10 data,
    labels for X
    :return: X_p, y_p
        X_p is a numpy.ndarray containing the preprocessed X
        y_p is a numpy.ndarray containing the preprocessed y
    """
    X_p = tf.keras.applications.vgg16.preprocess_input(X)
    y_p = tf.keras.utils.to_categorical(y, 10)
    return X_p, y_p

X_train_vgg16, y_train_vgg16 = preprocess_data(X_train, y_train)
X_test_vgg16, y_test_vgg16 = preprocess_data(X_test, y_test)

nb_train_samples = X_train_vgg16.shape[0]
nb_validation_samples = X_test_vgg16.shape[0]
input_tensor=tf.keras.Input(shape=X_train_vgg16.shape[1:])

In [17]:
base_vgg = tf.keras.applications.vgg16.VGG16(weights='imagenet', include_top=False, input_shape=INPUT_SHAPE)
# Extract the last layer from third block of vgg16 model
last = base_vgg.get_layer('block3_pool').output
# Add classification layers on top of it
x = Flatten()(last)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
pred = Dense(10, activation='sigmoid')(x)

vggNet = Model(base_vgg.input, pred)

# set the base model's layers to non-trainable
# uncomment next two lines if you don't want to
# train the base model
# for layer in base_vgg.layers:
#     layer.trainable = False

In [18]:
# compile the model with a SGD/momentum optimizer
# and a very slow learning rate.
vggNet.compile(loss='binary_crossentropy',
              optimizer=tf.keras.optimizers.SGD(learning_rate=LR, momentum=MOMENTUM),
              metrics=['accuracy'])

In [19]:
vggNet.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 32, 32, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 32, 32, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 16, 16, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 16, 16, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 16, 16, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 8, 8, 128)         0   

In [20]:
# fine-tune the model
vggNet.fit(
    # train_generator,
    X_train_vgg16,
    y_train_vgg16,
    steps_per_epoch=nb_train_samples,
    epochs=EPOCHS,
    # validation_data=validation_generator,
    validation_data=(X_test_vgg16, y_test_vgg16),
    validation_steps=nb_validation_samples)

Epoch 1/10
50000/50000 [==============================] - 1296s 26ms/step - loss: nan - accuracy: 0.1000 - val_loss: nan - val_accuracy: 0.1000
Epoch 2/10
50000/50000 [==============================] - 1585s 32ms/step - loss: nan - accuracy: 0.1000 - val_loss: nan - val_accuracy: 0.1000
Epoch 3/10
50000/50000 [==============================] - 1231s 25ms/step - loss: nan - accuracy: 0.1000 - val_loss: nan - val_accuracy: 0.1000
Epoch 4/10
50000/50000 [==============================] - 1281s 26ms/step - loss: nan - accuracy: 0.1000 - val_loss: nan - val_accuracy: 0.1000
Epoch 5/10
50000/50000 [==============================] - 1334s 27ms/step - loss: nan - accuracy: 0.1000 - val_loss: nan - val_accuracy: 0.1000
Epoch 6/10
50000/50000 [==============================] - 1270s 25ms/step - loss: nan - accuracy: 0.1000 - val_loss: nan - val_accuracy: 0.1000
Epoch 7/10
50000/50000 [==============================] - 1200s 24ms/step - loss: nan - accuracy: 0.1000 - val_loss: nan - val_accuracy:

## Models Evaluation

### Custom Models

In [27]:
y_actual_test_classes = np.argmax(y_test_ohe, axis=-1)

#### Model concept no. 1

In [39]:
y_pred_test_1 = model1.predict(X_test_norm)
y_pred_test_classes_1 = np.argmax(y_pred_test_1, axis=-1)

In [40]:
pd.DataFrame(confusion_matrix(y_actual_test_classes, y_pred_test_classes_1))

,0,1,2,3,4,5,6,7,8,9
0,837,10,40,14,13,2,9,6,49,20
1,6,912,1,2,1,1,4,2,12,59
2,47,2,698,36,79,49,61,17,9,2
3,19,6,59,630,68,123,58,17,9,11
4,7,2,42,42,830,21,29,21,5,1
5,14,3,42,135,44,717,17,20,3,5
6,4,1,32,36,24,16,876,3,5,3
7,8,3,24,30,53,29,5,841,2,5
8,46,17,6,5,4,4,5,5,888,20
9,22,44,4,7,1,3,4,7,17,891


In [41]:
print(classification_report(y_actual_test_classes, y_pred_test_classes_1))

              precision    recall  f1-score   support

           0       0.83      0.84      0.83      1000
           1       0.91      0.91      0.91      1000
           2       0.74      0.70      0.72      1000
           3       0.67      0.63      0.65      1000
           4       0.74      0.83      0.78      1000
           5       0.74      0.72      0.73      1000
           6       0.82      0.88      0.85      1000
           7       0.90      0.84      0.87      1000
           8       0.89      0.89      0.89      1000
           9       0.88      0.89      0.88      1000

    accuracy                           0.81     10000
   macro avg       0.81      0.81      0.81     10000
weighted avg       0.81      0.81      0.81     10000



#### Model concept no. 2

In [42]:
y_pred_test_2 = model2.predict(X_test_norm)
y_pred_test_classes_2 = np.argmax(y_pred_test_2, axis=-1)

In [43]:
pd.DataFrame(confusion_matrix(y_actual_test_classes, y_pred_test_classes_2))

,0,1,2,3,4,5,6,7,8,9
0,849,9,38,13,10,0,5,10,41,25
1,4,920,1,3,0,3,4,1,10,54
2,48,1,723,40,60,45,60,14,6,3
3,15,4,51,644,47,136,67,17,8,11
4,10,1,39,45,807,22,40,29,4,3
5,7,2,23,121,40,761,16,26,1,3
6,5,0,17,33,14,8,915,1,5,2
7,10,1,18,29,39,41,5,852,1,4
8,33,15,9,5,3,2,6,3,910,14
9,12,37,3,9,2,1,4,4,12,916


In [44]:
print(classification_report(y_actual_test_classes, y_pred_test_classes_2))

              precision    recall  f1-score   support

           0       0.85      0.85      0.85      1000
           1       0.93      0.92      0.92      1000
           2       0.78      0.72      0.75      1000
           3       0.68      0.64      0.66      1000
           4       0.79      0.81      0.80      1000
           5       0.75      0.76      0.75      1000
           6       0.82      0.92      0.86      1000
           7       0.89      0.85      0.87      1000
           8       0.91      0.91      0.91      1000
           9       0.89      0.92      0.90      1000

    accuracy                           0.83     10000
   macro avg       0.83      0.83      0.83     10000
weighted avg       0.83      0.83      0.83     10000



#### Model concept no. 3

In [45]:
y_pred_test_3 = model3.predict(X_test_norm)
y_pred_test_classes_3 = np.argmax(y_pred_test_3, axis=-1)

In [46]:
pd.DataFrame(confusion_matrix(y_actual_test_classes, y_pred_test_classes_3))

,0,1,2,3,4,5,6,7,8,9
0,669,37,72,18,19,12,15,19,106,33
1,36,764,3,12,8,8,20,10,28,111
2,75,9,402,87,140,108,98,36,27,18
3,24,20,66,391,76,225,114,53,19,12
4,39,4,90,47,538,71,93,100,12,6
5,10,5,65,204,75,507,43,69,10,12
6,2,6,41,74,66,25,763,10,8,5
7,24,5,29,44,74,116,18,658,1,31
8,99,48,22,20,12,6,9,3,739,42
9,38,127,9,20,7,9,16,29,51,694


In [47]:
print(classification_report(y_actual_test_classes, y_pred_test_classes_3))

              precision    recall  f1-score   support

           0       0.66      0.67      0.66      1000
           1       0.75      0.76      0.75      1000
           2       0.50      0.40      0.45      1000
           3       0.43      0.39      0.41      1000
           4       0.53      0.54      0.53      1000
           5       0.47      0.51      0.49      1000
           6       0.64      0.76      0.70      1000
           7       0.67      0.66      0.66      1000
           8       0.74      0.74      0.74      1000
           9       0.72      0.69      0.71      1000

    accuracy                           0.61     10000
   macro avg       0.61      0.61      0.61     10000
weighted avg       0.61      0.61      0.61     10000



#### Residual Network (ResNet)

In [53]:
# TODO
resNet.evaluate(X_test_resnet50, y_test_resnet50)

313/313 [==============================] - 21s 69ms/step - loss: 1.1916 - accuracy: 0.6716


[1.1915991306304932, 0.6715999841690063]

In [54]:
y_pred_test_resnet = resNet.predict(X_test_resnet50)
y_pred_test_classes_resnet = np.argmax(y_pred_test_resnet, axis=-1)

In [55]:
pd.DataFrame(confusion_matrix(y_actual_test_classes, y_pred_test_classes_resnet))

,0,1,2,3,4,5,6,7,8,9
0,718,32,45,11,23,9,7,17,93,45
1,27,736,5,14,9,20,9,11,47,122
2,62,12,582,54,101,53,83,33,12,8
3,17,32,63,492,64,170,85,41,13,23
4,14,3,67,56,631,46,84,80,13,6
5,7,14,45,160,53,611,45,52,4,9
6,10,12,60,55,66,47,730,7,6,7
7,23,16,24,50,63,66,15,708,9,26
8,72,52,19,10,13,8,0,6,772,48
9,45,108,1,15,9,12,8,23,43,736


In [56]:
print(classification_report(y_actual_test_classes, y_pred_test_classes_resnet))

              precision    recall  f1-score   support

           0       0.72      0.72      0.72      1000
           1       0.72      0.74      0.73      1000
           2       0.64      0.58      0.61      1000
           3       0.54      0.49      0.51      1000
           4       0.61      0.63      0.62      1000
           5       0.59      0.61      0.60      1000
           6       0.68      0.73      0.71      1000
           7       0.72      0.71      0.72      1000
           8       0.76      0.77      0.77      1000
           9       0.71      0.74      0.73      1000

    accuracy                           0.67     10000
   macro avg       0.67      0.67      0.67     10000
weighted avg       0.67      0.67      0.67     10000



#### VGG16

In [21]:
# TODO
vggNet.evaluate(X_test_vgg16, y_test_vgg16)

313/313 [==============================] - 54s 172ms/step - loss: nan - accuracy: 0.1000


[nan, 0.10000000149011612]

In [22]:
y_pred_test_vgg16 = vggNet.predict(X_test_vgg16)
y_pred_test_classes_vgg16 = np.argmax(y_pred_test_vgg16, axis=-1)

In [28]:
pd.DataFrame(confusion_matrix(y_actual_test_classes, y_pred_test_classes_vgg16))

,0,1,2,3,4,5,6,7,8,9
0,1000,0,0,0,0,0,0,0,0,0
1,1000,0,0,0,0,0,0,0,0,0
2,1000,0,0,0,0,0,0,0,0,0
3,1000,0,0,0,0,0,0,0,0,0
4,1000,0,0,0,0,0,0,0,0,0
5,1000,0,0,0,0,0,0,0,0,0
6,1000,0,0,0,0,0,0,0,0,0
7,1000,0,0,0,0,0,0,0,0,0
8,1000,0,0,0,0,0,0,0,0,0
9,1000,0,0,0,0,0,0,0,0,0


In [29]:
print(classification_report(y_actual_test_classes, y_pred_test_classes_vgg16))

              precision    recall  f1-score   support

           0       0.10      1.00      0.18      1000
           1       0.00      0.00      0.00      1000
           2       0.00      0.00      0.00      1000
           3       0.00      0.00      0.00      1000
           4       0.00      0.00      0.00      1000
           5       0.00      0.00      0.00      1000
           6       0.00      0.00      0.00      1000
           7       0.00      0.00      0.00      1000
           8       0.00      0.00      0.00      1000
           9       0.00      0.00      0.00      1000

    accuracy                           0.10     10000
   macro avg       0.01      0.10      0.02     10000
weighted avg       0.01      0.10      0.02     10000



C:\Users\ejowik001\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ejowik001\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ejowik001\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
